# Curve-fit to estimate final dissipation

In [1]:
%run base.py
%run paths.py

In [2]:
from base import *
from paths import *

In [3]:
%matplotlib ipympl
import matplotlib.pyplot as plt

def get_teps(short_name):
    path = paths_sim[short_name]
    d = SpatialMeansSW1L._load(path)
    t = d['t']
    eps = d['epsK'] + d ['epsA']
    
    idx = _index_where(eps, eps.max())
    
    return t, eps, idx

def plot_eps(short_name):
    t, eps,idx = get_teps(short_name)
    plt.plot(t, eps, label=short_name)
    plt.text(t[idx], eps[idx], short_name)
    # plt.legend()

## Some extreme cases
#plot_eps('noise_c400nh7680Buinf')
#plot_eps('vortex_grid_c100nh1920Bu2.0efr1.00e+00')
#plot_eps('noise_c20nh7680Buinf')
#plot_eps('vortex_grid_c100nh1920Bu1.0efr1.00e+00')
#plot_eps('noise_c40nh7680Buinf')
#plot_eps('vortex_grid_c100nh1920Buinfefr1.00e+02')

## Lognorm like
# plot_eps('vortex_grid_c20nh1920Buinfefr1.00e+01')
# plot_eps('vortex_grid_c100nh1920Buinfefr1.00e+02')
# plot_eps('vortex_grid_c100nh1920Bu4.0efr1.00e+02')
# plot_eps('vortex_grid_c400nh1920Buinfefr1.00e+02')

#for short in df_vort['short name']:
#    plot_eps(short)

plot_eps('noise_c400nh3840Buinf')
# plot_eps('noise_c100nh3840Buinf')
# plot_eps('vortex_grid_c100nh1920Bu2.0efr1.00e+00')

FigureCanvasNbAgg()

In [4]:
%matplotlib ipympl
from scipy.signal import medfilt
from scipy.special import erf

t, eps, idx = get_teps("vortex_grid_c400nh1920Buinfefr1.00e+02")
# plt.figure()
# plt.plot(t, np.tanh(t/10))
# plt.plot(t, np.tanh((t/10)**4))
# plt.plot(t, erf((t/10)**4))
# plt.show()

In [5]:
eps_filt = medfilt(eps, 7)
plt.figure()
plt.plot(t, eps)
plt.plot(t, eps_filt)

FigureCanvasNbAgg()

# Curve fit

In [6]:
%matplotlib ipympl
from scipy.optimize import curve_fit
from scipy.signal import lti, step2
from scipy import stats
from matplotlib.pyplot import *

option = 2

# short = 'vortex_grid_c100nh1920Buinfefr1.00e-01'
# short = 'vortex_grid_c100nh1920Buinfefr1.00e+02'
# short = 'vortex_grid_c20nh1920Buinfefr1.00e+01'
# short = 'vortex_grid_c400nh1920Buinfefr1.00e+02'
# short = 'noise_c20nh7680Buinf'
# short = 'vortex_grid_c20nh960Buinfefr1.00e+00'
# short = 'noise_c20nh3840Buinf'
# short = 'noise_c400nh3840Buinf'
short = 'vortex_grid_c20nh1920Bu4.0efr1.00e-02'
short = 'vortex_grid_c20nh1920Bu20.0efr1.00e+00'
short = 'noise_c400nh1920Buinf'
# short = 'vortex_grid_c100nh1920Bu4.0efr1.00e+02'
# short = 'vortex_grid_c100nh1920Bu2.0efr1.00e+00'

t, eps,_ = get_teps(short)

if option == 1:
    def f(x, amptan, ttan):
        return amptan * pl.tanh(2 * (x / ttan)**4)

    guesses = [pl.median(eps), t[eps==eps.max()]]
else:
    # def f(x, amptan, ttan, amplog, tlog, sigma):
    def f(x, amptan, ttan, amplog, sigma):
        return  (
            amptan * np.tanh(2 * (x/ttan)**4) +
            amplog * stats.lognorm.pdf(x, scale=np.exp(ttan), s=sigma)
        )


    guesses = {
        'amptan': np.median(eps),
        'ttan': t[eps==eps.max()],
        'amplog': eps.max(),
        # 'tlog': t[eps==eps.max()],
        'sigma': eps.std()
    }
    guesses = np.array(list(guesses.values()), dtype=float)
    bounds = (0, guesses * 1.5)

# popt, pcov = curve_fit(f, t, eps)
# popt, pcov = curve_fit(f, t, eps, sigma=1./t)
popt, pcov = curve_fit(f, t, eps, guesses)

# popt, pcov = curve_fit(f, t, eps, guesses, bounds=bounds, method="trf")

plot(t, eps, label='original')
plot(t, f(t, *popt), label='curve fit')
plot(t, np.median(eps) * np.ones_like(eps), 'g', label='median_all')

plot(t, np.median(eps[t>40]) * np.ones_like(eps), 'r:', label='median')
plot(t, np.mean(eps[t>40]) * np.ones_like(eps), 'r--', label='mean')

# df = df_vort if  'vortex' in short else df_noise
# eps_chosen = get_row(df, 'short name', short)['$\epsilon$'].iloc[0]

# plot(t, eps_chosen * np.ones_like(eps), 'k', label='chosen')
# plot(t, popt[2] * stats.lognorm.pdf(t, *popt[-2:]), label='lognorm')
legend()

/scratch/avmo/opt/fluidmeta-MmQYGCPX/lib/python3.5/site-packages/scipy/optimize/minpack.py:794: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


FigureCanvasNbAgg()

In [7]:
eps_fit = f(t, *popt)
dt = t[1]-t[0]
# dt = np.median(np.gradient(t))
deps_fit = np.gradient(eps_fit, dt)
ddeps_fit = np.gradient(deps_fit, dt)
curv = ddeps_fit / (1 + deps_fit) ** 1.5
# curv = curv*eps.max()/curv.max()

In [8]:
figure()
plot(t, eps_fit)
plot(t, curv)
# plot(t, deps_fit)

FigureCanvasNbAgg()

### Kneedle algorithm

In [9]:
def locate_knee(time, eps_fit, eps_stat):
    from kneed import KneeLocator

    while not np.array_equal(time, np.sort(time)):
        idx_del = np.where(np.diff(time) < 0)[0] + 1
        time = np.delete(time, idx_del)
        eps_fit = np.delete(eps_fit, idx_del)
    
    if eps_fit.max() > 2 * eps_stat:
        # log-norm + tanh
        knee = KneeLocator(time, eps_fit, direction='decreasing')
        idx = knee.knee_x
    else:
        knee = KneeLocator(time, eps_fit)
        idx = knee.knee_x

    if idx is None:
        # non-stationary case
        idx = -1

    time_stat = time[idx]
    return time_stat

locate_knee(t, eps_fit, eps_fit[-1])

21.30833

In [10]:
from kneed import KneeLocator


while not np.array_equal(t, np.sort(t)):
    idx_del = np.where(np.diff(t) < 0)[0] + 1
    t = np.delete(t, idx_del)
    eps_fit = np.delete(eps_fit, idx_del)
    print(idx_del)
    

knee = KneeLocator(t, eps_fit)
knee.plot_knee()

FigureCanvasNbAgg()

In [11]:
t[knee.knee_x], knee.direction

(21.30833, 'increasing')

In [12]:
%matplotlib
np.where(np.gradient(t) <= 0)[0]
# plt.plot(t, np.gradient(t))

Using matplotlib backend: module://ipympl.backend_nbagg


array([], dtype=int64)

In [13]:
idx_neq = np.where(t != np.sort(t))[0]
print(idx_neq)
print(t[idx_neq])

[]
[]


In [14]:
idx = _index_where(eps_fit, np.median(eps)); t[idx]

19.70729

In [15]:
idx = _index_where(abs(curv), 1e-5); t[idx]

0.2001302

In [16]:
idx = np.argmin(abs(curv)); t[idx]

33.81647

###  Histogram of curvatures

In [17]:
curv.std()*0.01

5.9983221272149785e-05

In [18]:
%matplotlib ipympl
n, bins, patches = plt.hist(curv, 10, normed=1, facecolor='green', alpha=0.75)

FigureCanvasNbAgg()

/scratch/avmo/opt/fluidmeta-MmQYGCPX/lib/python3.5/site-packages/matplotlib-2.2.2-py3.5-linux-x86_64.egg/matplotlib/axes/_axes.py:6462: UserWarning: The 'normed' kwarg is deprecated, and has been replaced by the 'density' kwarg.
  warnings.warn("The 'normed' kwarg is deprecated, and has been "


In [19]:
idx = _index_flat(eps_fit, t, 1e-4); t[idx]

TypeError: _index_flat() takes from 1 to 2 positional arguments but 3 were given

In [20]:
popt

array([ 1.15158681, 19.09638869,  0.33740296,  1.4558777 ])

# Cumulative average

In [21]:
from fluidsim.base.output.spect_energy_budget import cumsum_inv
import numpy as np


def cummean(x):
    """Cumulative average from the reversed array."""
    sum_fwd = x.cumsum()
    idx_fwd = np.arange(1, x.shape[0]+1)
    return sum_fwd / idx_fwd


def cummean_inv(x):
    """Cumulative average from the reversed array."""
    sum_inv = cumsum_inv(x)
    idx_inv = np.arange(x.shape[0], 0, -1)
    return sum_inv / idx_inv

In [22]:
eps_mean = cummean(eps)
eps_mean_inv = cummean_inv(eps)

plt.figure()
plt.plot(t, eps)
plt.plot(t, eps_mean)
plt.plot(t, eps_mean_inv)

FigureCanvasNbAgg()

# Moving average (from SciPy cookbook)

In [23]:
import numpy

def smooth(x,window_len=11,window='hanning'):
    """smooth the data using a window with requested size.
    
    This method is based on the convolution of a scaled window with the signal.
    The signal is prepared by introducing reflected copies of the signal 
    (with the window size) in both ends so that transient parts are minimized
    in the begining and end part of the output signal.
    
    input:
        x: the input signal 
        window_len: the dimension of the smoothing window; should be an odd integer
        window: the type of window from 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'
            flat window will produce a moving average smoothing.

    output:
        the smoothed signal
        
    example:

    t=linspace(-2,2,0.1)
    x=sin(t)+randn(len(t))*0.1
    y=smooth(x)
    
    see also: 
    
    numpy.hanning, numpy.hamming, numpy.bartlett, numpy.blackman, numpy.convolve
    scipy.signal.lfilter
 
    TODO: the window parameter could be the window itself if an array instead of a string
    NOTE: length(output) != length(input), to correct this: return y[(window_len/2-1):-(window_len/2)] instead of just y.
    """

    if x.ndim != 1:
        raise ValueError("smooth only accepts 1 dimension arrays.")

    if x.size < window_len:
        raise ValueError("Input vector needs to be bigger than window size.")


    if window_len<3:
        return x


    if not window in ['flat', 'hanning', 'hamming', 'bartlett', 'blackman']:
        raise ValueError("Window is on of 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'")


    s=numpy.r_[x[window_len-1:0:-1],x,x[-2:-window_len-1:-1]]
    #print(len(s))
    if window == 'flat': #moving average
        w=numpy.ones(window_len,'d')
    else:
        w=eval('numpy.'+window+'(window_len)')

    y=numpy.convolve(w/w.sum(),s,mode='valid')
    return y

In [24]:
from scipy.stats import linregress
linregress(t[100:200], eps[100:200])

LinregressResult(slope=0.06070906172579099, intercept=-0.21404717186330946, rvalue=0.953253264131833, pvalue=9.784940094962523e-53, stderr=0.0019439590264217056)

In [29]:
def f(x, a):
    return a * np.ones_like(x)

curve_fit(f, t[300:], eps[300:])

(array([1.17130666]), array([[8.75858778e-06]]))

In [30]:
eps[300:].mean()

1.1713066614015573

In [31]:
plt.figure()
plt.plot(eps)

FigureCanvasNbAgg()

In [32]:
eps_filt = medfilt(eps, 51)
eps_mavg = smooth(eps, 21)[20:]
plt.figure()
plt.plot(t, eps)
plt.plot(t, eps_filt, label="median filtered")
plt.plot(t, eps_mavg, label="averaged")
plt.legend()

FigureCanvasNbAgg()

# Using FFT

In [33]:
Ts = t[1] - t[0];  # sampling interval
Fs = 1.0/Ts; # sampling rate
tvec = t # time vector

y = eps

n = len(y) # length of the signal
k = np.arange(n)
T = n/Fs
frq = k/T # two sides frequency range
frq = frq[0:n//2] # one side frequency range

Y = np.fft.fft(y)/n # fft computing and normalization
Y = Y[0:n//2]
plt.clf()
fig, ax = plt.subplots(2, 1)
ax[0].plot(tvec,y)
ax[0].set_xlabel('Time')
ax[0].set_ylabel('Amplitude')
ax[1].loglog(frq[1:],abs(Y[1:]),'r') # plotting the spectrum
ax[1].set_xlabel('Freq (Hz)')
ax[1].set_ylabel('|Y(freq)|')


FigureCanvasNbAgg()

Text(0,0.5,'|Y(freq)|')

# Peak detection

In [34]:
%matplotlib ipympl
from matplotlib.pyplot import *
from scipy.signal import find_peaks_cwt, find_peaks

widths = np.diff(t)
peaks = find_peaks(eps)[0]
plot(t, eps)
scatter(t[peaks], eps[peaks])

FigureCanvasNbAgg()

In [37]:
eps[-1], eps_filt[-1]

(1.0429323, 1.0429323)

In [38]:
def step_info(t, yout, thresh_percent=20):
    thresh = 1 + thresh_percent / 100
    result = dict(
        overshoot_percent=(yout.max() / yout[-1] - 1) * 100,
        rise_time=(
            t[next(i for i in range(0,len(yout)-1)
                   if yout[i]>yout[-1]*.90)]
            - t[0]
        ),
        settling_time=(
            t[next(len(yout)-i for i in range(2,len(yout)-1)
                   if abs(yout[-i]/yout[-1])>thresh)]
            - t[0]
        ),
    )
    return result

step_info(t, eps_filt)

{'overshoot_percent': 27.94490112157808,
 'rise_time': 19.4071,
 'settling_time': 110.6395}

In [39]:
yout = eps
thresh_settling = 1.20
idx = np.where(np.abs(yout / yout[-1]) > thresh_settling)[0][-1]
settling_time = t[idx] - t[0]
settling_time == step_info(t, yout)["settling_time"]

True